<a href="https://colab.research.google.com/github/schase15/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Steven_Chase_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [34]:
# Look at the dataset
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Create new features

# Luxury items in building
df['luxury'] = df['elevator'] + df['doorman'] + df['fitness_center']

# Area of NYC, ratio between latitude and longitude
df['area'] = df['latitude'] / df['longitude']

# Total number of rooms
df['rooms'] = df['bathrooms'] + df['bedrooms']

In [114]:
# Look at new features

df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,luxury,area,area_2,rooms
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.550624,-1.816122,0.50
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,-0.551528,-1.813145,0.50
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,-0.550511,-1.816494,1.00
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.550969,-1.814985,1.00
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.552055,-1.811413,0.25


In [0]:
# Split into test and train
# Train data is from April and May of 2016
# Test data is from June 2016

train = df[df['created'] < '2016-06-1 00:00:00']
test = df[df['created'] >= '2016-06-1 00:00:00']

In [78]:
# Start with a baseline model
from sklearn.metrics import mean_absolute_error

# Define target
target= ['price']

# Arrange y target vectors for train and test data
y_train= train[target]
y_test= test[target]

# Create y predictions
guess = y_train.mean()

# Calculate the mae for our baseline model for Train data
## Prediciton array for train
y_pred_train = [guess] * len(y_train)

# Mae result
mae= mean_absolute_error(y_train, y_pred_train)
print('Train error with baseline model is', mae)

# Calculate the mae for our baseline model for Test data
## Prediciton array for test
y_pred_test = [guess] * len(y_test)

# Mae result
mae= mean_absolute_error(y_test, y_pred_test)
print('Test error with baseline model is', mae)

Train error with baseline model is 1200.6979788395183
Test error with baseline model is 1203.1224507386557


In [227]:
# Multiple regression

# Import model and make an instance
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Define features and target
features= ['rooms', 'luxury', 'area']
target= ['price']

# Create target vectors
y_train= train[target]
y_test= test[target]

# Create feature matrices
X_train= train[features]
X_test= test[features]

# Fit the model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [228]:
# # Calculate and report the error metrics: MSE, RMSE, MAE and R^2

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# TRAINING DATA
# Make y predictions
y_pred_train = model.predict(X_train)

# Mse, rmse, mae, r^2
mse_train= mean_squared_error(y_train, y_pred_train)
rmse_train= np.sqrt(mse_train)
mae_train= mean_absolute_error(y_train, y_pred_train)
r2_train= r2_score(y_train, y_pred_train)

print('Train Mean Squared Error:', mse_train)
print('Train Root Mean Squared Error:', rmse_train)
print('Train Mean Absolute Error:', mae_train)
print('Train R^2:', r2_train)

Train Mean Squared Error: 1552822.1952942975
Train Root Mean Squared Error: 1246.1228652481655
Train Mean Absolute Error: 805.0577910509851
Train R^2: 0.4990739643604619


In [229]:
# TESTING DATA
# Make y predictions
y_pred_test= model.predict(X_test)

# Mse, rmse, mae, r^2
mse_test= mean_squared_error(y_test, y_pred_test)
rmse_test= np.sqrt(mse_test)
mae_test= mean_absolute_error(y_test, y_pred_test)
r2_test= r2_score(y_test, y_pred_test)

print('Test Mean Squared Error:', mse_test)
print('Test Root Mean Squared Error:', rmse_test)
print('Test Mean Absolute Error:', mae_test)
print('Test R^2:', r2_test)

Test Mean Squared Error: 1602031.6906704279
Test Root Mean Squared Error: 1265.7139055372775
Test Mean Absolute Error: 830.9224271753699
Test R^2: 0.4875675592505738


In [230]:
# Model's coefficients and interecept

print('Intercept', model.intercept_[0])
coefficients = pd.Series(model.coef_[0], features)
print(coefficients.to_string())

Intercept 145546.8173503807
rooms        802.229961
luxury       355.122090
area      262490.203748


In [231]:
# Standardize to look at the impact each feature had on the predicted target

from sklearn.preprocessing import StandardScaler

# Instantize
scaler= StandardScaler()


X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)
model.fit(X_train_scaled, y_train)

# Interpret model fit
print('Intercept', model.intercept_[0])
coefficients = pd.Series(model.coef_[0], features)
print(coefficients.to_string())

Intercept 3578.6241538914514
rooms     1133.030299
luxury     427.866985
area       163.799345
